In [3]:
#!/usr/bin/env python3
import numpy as np
import plotly.graph_objects as go

# Sphere and shell parameters
r_inner = 1.0   # Inner sphere radius
r_outer = 1.03   # Outer shell radius

# Create mesh for spherical surfaces
n_theta = 180   # colatitude resolution
n_phi = 360     # longitude resolution

theta = np.linspace(0, np.pi, n_theta)
phi = np.linspace(0, 2*np.pi, n_phi)
theta_grid, phi_grid = np.meshgrid(theta, phi)

# Define wedge cutout parameters (in upper hemisphere)
# Remove 1/8 wedge: 45 degrees in phi, upper hemisphere only
wedge_phi_min = 0       # degrees
wedge_phi_max = 45      # degrees (1/8 of 360)
wedge_phi_min_rad = np.radians(wedge_phi_min)
wedge_phi_max_rad = np.radians(wedge_phi_max)

# ========================================
# INNER SPHERE (full sphere at r=1.0)
# ========================================
x_inner = r_inner * np.sin(theta_grid) * np.cos(phi_grid)
y_inner = r_inner * np.sin(theta_grid) * np.sin(phi_grid)
z_inner = r_inner * np.cos(theta_grid)

# ========================================
# OUTER SHELL (with wedge cutout)
# ========================================
x_outer = r_outer * np.sin(theta_grid) * np.cos(phi_grid)
y_outer = r_outer * np.sin(theta_grid) * np.sin(phi_grid)
z_outer = r_outer * np.cos(theta_grid)

# Create mask for wedge cutout
# Remove wedge where: z > 0 AND phi is between wedge_phi_min and wedge_phi_max
wedge_mask = (z_outer > 0) & (phi_grid >= wedge_phi_min_rad) & (phi_grid <= wedge_phi_max_rad)

# Apply mask: set cutout region to NaN
x_outer_cut = np.where(wedge_mask, np.nan, x_outer)
y_outer_cut = np.where(wedge_mask, np.nan, y_outer)
z_outer_cut = np.where(wedge_mask, np.nan, z_outer)

# ========================================
# CREATE PLOTLY FIGURE
# ========================================
fig = go.Figure()

# Add inner sphere (blue)
fig.add_trace(go.Surface(
    x=x_inner,
    y=y_inner,
    z=z_inner,
    colorscale=[[0, 'lightblue'], [1, 'lightblue']],
    showscale=False,
    name='Mercury',
    showlegend=True,
    opacity=1.0,
))

# Add outer shell with cutout (orange/coral)
fig.add_trace(go.Surface(
    x=x_outer_cut,
    y=y_outer_cut,
    z=z_outer_cut,
    colorscale=[[0, 'coral'], [1, 'coral']],
    showscale=False,
    name='Integration Shell',
    showlegend=True,
    opacity=0.95,
))

# ========================================
# ADD CUT EDGES (optional - makes cutout clearer)
# ========================================
# Create lines along the edges of the cutout
edge_theta = np.linspace(0, np.pi/2, 50)  # Upper hemisphere only

# Edge 1: phi = wedge_phi_min_rad
x_edge1 = r_outer * np.sin(edge_theta) * np.cos(wedge_phi_min_rad)
y_edge1 = r_outer * np.sin(edge_theta) * np.sin(wedge_phi_min_rad)
z_edge1 = r_outer * np.cos(edge_theta)

fig.add_trace(go.Scatter3d(
    x=x_edge1, y=y_edge1, z=z_edge1,
    mode='lines',
    line=dict(color='darkred', width=4),
    name='Cut Edge 1',
    showlegend=False,
    hoverinfo='skip'
))

# Edge 2: phi = wedge_phi_max_rad
x_edge2 = r_outer * np.sin(edge_theta) * np.cos(wedge_phi_max_rad)
y_edge2 = r_outer * np.sin(edge_theta) * np.sin(wedge_phi_max_rad)
z_edge2 = r_outer * np.cos(edge_theta)

fig.add_trace(go.Scatter3d(
    x=x_edge2, y=y_edge2, z=z_edge2,
    mode='lines',
    line=dict(color='darkred', width=4),
    name='Cut Edge 2',
    showlegend=False,
    hoverinfo='skip'
))

# Edge 3: theta = pi/2 (equator boundary of cutout)
edge_phi = np.linspace(wedge_phi_min_rad, wedge_phi_max_rad, 50)
x_edge3 = r_outer * np.cos(edge_phi)
y_edge3 = r_outer * np.sin(edge_phi)
z_edge3 = np.zeros_like(edge_phi)

fig.add_trace(go.Scatter3d(
    x=x_edge3, y=y_edge3, z=z_edge3,
    mode='lines',
    line=dict(color='darkred', width=4),
    name='Cut Edge 3',
    showlegend=False,
    hoverinfo='skip'
))

# ========================================
# LAYOUT AND CAMERA
# ========================================
fig.update_layout(
    title=dict(
        text='Mercury (Rₘ=2440.0 km)<br>Simulation grid cell size 75 km ⇒ dR = 40 km',
        x=0.5,
        xanchor='center',
        font=dict(size=18)
    ),
    scene=dict(
        xaxis=dict(title='X [Rₘ]', range=[-1.2, 1.2], showgrid=True, gridcolor='lightgray'),
        yaxis=dict(title='Y [Rₘ]', range=[-1.2, 1.2], showgrid=True, gridcolor='lightgray'),
        zaxis=dict(title='Z [Rₘ]', range=[-1.2, 1.2], showgrid=True, gridcolor='lightgray'),
        aspectmode='cube',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.2),  # View from upper-right to see cutout
            center=dict(x=0, y=0, z=0)
        ),
        bgcolor='white'
    ),
    width=900,
    height=800,
    showlegend=True,
    legend=dict(x=0.7, y=0.9, bgcolor='rgba(255,255,255,0.8)'),
    template='plotly_white'
)

# Save to HTML file
# fig.write_html('sphere_shell_cutaway.html')
# print("Saved: sphere_shell_cutaway.html")

# Display in browser
fig.show()
